## Feature Engineering and CV based on 5-th Place Solutions

`param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}`
         
`n_repeats=2
n_trees = 150`

val-MAP@7:0.89935, private LB: 0.0266884, public LB: 0.0264044

New in this notebook:
- Create training data for all months (2015-02-28 to 2016-04-28) and validate on 2016-05-28

In [1]:
from santander_helper import *

Create all months' data

In [2]:
# fixed_lag = 6
# for i, m in tqdm.tqdm_notebook(enumerate(month_list), total=len(month_list)):
#     print(m)
#     if m in ['2015-01-28', '2016-06-28']:
#         continue
#     x_train, y_train, w_train = create_train(m, max_lag=i, fixed_lag=fixed_lag, pattern_flag=True)
#     print('-'*60)
# del x_train, y_train, w_train
# gc.collect()

Load all months' data

In [3]:
x_train = []
y_train = []
w_train = []
fixed_lag = 6
for i, m in tqdm.tqdm_notebook(enumerate(month_list), total=len(month_list)):
    if m in ['2015-01-28', '2016-06-28']:
        continue
    x_tmp, y_tmp, w_tmp = create_train(m, max_lag=i, fixed_lag=fixed_lag, pattern_flag=True)
    x_train.append(x_tmp)
    y_train.append(y_tmp)
    w_train.append(w_tmp)
del x_tmp, y_tmp, w_tmp
gc.collect()

# Prepare for train and validation
# x_val = x_train[-1]
# y_val = y_train[-1]
# w_val = w_train[-1]

x_train = pd.concat(x_train, axis=0, ignore_index=True, sort=False)
y_train = pd.concat(y_train, axis=0, ignore_index=True, sort=False)
w_train = pd.concat(w_train, axis=0, ignore_index=True, sort=False)

gc.collect()

217

In [ ]:
param = {'objective': 'multi:softprob', 
         'eta': 0.1, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'merror',
         'min_child_weight': 10,
         'lambda': 5,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}

n_rows = None # number of rows in train dataset, to simplify testing, always set to None
n_repeats = 3
n_trees = 50
train = {'x': x_train.iloc[:n_rows, :], 'y': y_train.iloc[:n_rows], 'w': w_train.iloc[:n_rows]}
# val = {'x': x_val.iloc[:n_rows, :], 'y': y_val.iloc[:n_rows], 'w': w_val.iloc[:n_rows]}
clfs, running_time = cv_all_month(param, train, n_features=350, num_boost_round=n_trees, 
    n_repeats=n_repeats, random_state=47, verbose_eval=True)

[0]	train-merror:0.265432	train-MAP@7:0
[1]	train-merror:0.262551	train-MAP@7:0


In [ ]:
x_test = create_test(month='2016-06-28', max_lag=17, fixed_lag=6, pattern_flag=True)

In [ ]:
simulation_name = 'p2'
y_pred, y_sub = predict_all_month(model_dict=clfs, x_test=x_test, 
    sub_name='eda_4_33_{}.csv.gz'.format(simulation_name), n_features=350, n_trees=n_trees)
save_pickle('eda_4_33_results_{}.pickle'.format(simulation_name), (clfs, running_time, y_pred, y_sub))

In [ ]:
y_pred2 = load_pickle('parameter_tune_eda_4_32_p4.pickle')
y_pred2 = y_pred2[-4]

y_pred_final = np.concatenate((y_pred, y_pred2))
y_pred_final = np.mean(y_pred_final, axis=0)

y_sub = np.argsort(y_pred_final, axis=1)
y_sub = np.fliplr(y_sub)[:, :7]
# Prepare submission
test_id = x_test.loc[:, 'ncodpers'].values
y_sub = [' '.join([target_cols[k] for k in pred]) for pred in y_sub]
y_sub = pd.DataFrame({'ncodpers': test_id, 'added_products': y_sub})
sub_name = 'eda_4_33_{}+eda_4_32_p4.csv.gz'.format(simulation_name)
y_sub.to_csv(sub_name, compression='gzip', index=False)